<a href="https://colab.research.google.com/github/fralfaro/MAT281_2022/blob/main/labs/lab_07/lab_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°07


<a id='p1'></a>
## I.- Problema 01


<img src="https://www.cardrates.com/wp-content/uploads/2020/08/shutterstock_576998230.jpg?1" width="480" height="360" align="center"/>


El conjunto de datos se denomina `creditcard.csv` y consta de varias columnas con información acerca del fraude de tarjetas de crédito, en donde la columna **Class** corresponde a: 0 si no es un fraude y 1 si es un fraude.

En este ejercicio se trabajará el problemas de  clases desbalancedas. Veamos las primeras cinco filas dle conjunto de datos:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
# cargar datos
df = pd.read_csv("https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/labs/lab_07/data/creditcard.csv",sep=';')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
2,10.0,1.249999,-1.221637,0.383930,-1.234899,-1.485419,-0.753230,-0.689405,-0.227487,-2.094011,...,-0.231809,-0.483285,0.084668,0.392831,0.161135,-0.354990,0.026416,0.042422,121.50,0
3,10.0,0.384978,0.616109,-0.874300,-0.094019,2.924584,3.317027,0.470455,0.538247,-0.558895,...,0.049924,0.238422,0.009130,0.996710,-0.767315,-0.492208,0.042472,-0.054337,9.99,0
4,12.0,1.103215,-0.040296,1.267332,1.289091,-0.735997,0.288069,-0.586057,0.189380,0.782333,...,-0.024612,0.196002,0.013802,0.103758,0.364298,-0.382261,0.092809,0.037051,12.99,0


Analicemos el total de fraudes respecto a los casos que nos son fraudes:


In [ ]:
# calcular proporciones
df_count = pd.DataFrame()
df_count["fraude"] =["no","si"]
df_count["total"] = df["Class"].value_counts() 
df_count["porcentaje"] = 100*df_count["total"] /df_count["total"] .sum()

df_count

,fraude,total,porcentaje
0,no,50000,99.025588
1,si,492,0.974412


Se observa que menos del 1% corresponde a registros frudulentos. La pregunta que surgen son:

* ¿ Cómo deben ser el conjunto de entrenamiento y de testeo?
* ¿ Qué modelos ocupar?
* ¿ Qué métricas ocupar?

Por ejemplo, analicemos el modelos de regresión logística y apliquemos el procedimiento estándar:

In [ ]:
# datos 
y = df.Class
X = df.drop('Class', axis=1)

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)


# Creando el modelo
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# predecir
lr_pred = lr.predict(X_test)

# calcular accuracy
accuracy_score(y_test, lr_pred)

0.9961181969420898

En general el modelo tiene un **accuracy** del 99,9%, es decir, un podría suponer que el modelo predice casi perfectamente, pero eso esta lejos de ser así.  Para ver por qué es necesario seguir los siguientes pasos:

### 1. Cambiar la métrica de rendimiento

El primer paso es comparar con distintas métricas, para eso ocupemos las 4 métricas clásicas abordadas en el curso:
* accuracy
* precision
* recall
* f-score

En este punto deberá poner las métricas correspondientes y comentar sus resultados.

In [ ]:
# metrics
y_true =  list(y_test)
y_pred = list(lr.predict(X_test))

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_test, lr_pred))
print('recall:     ',recall_score(y_test, lr_pred))
print('precision:  ',precision_score(y_test, lr_pred))
print('f-score:    ',f1_score(y_test, lr_pred))
print("")


Matriz de confusion:
 
[[12471    16]
 [   33   103]]

Metricas:
 
accuracy:    0.9961181969420898
recall:      0.7573529411764706
precision:   0.865546218487395
f-score:     0.807843137254902



Matriz de confusión : Es una matriz en cuyas filas y columnas estan las clases observadas y predichas respectivamente. Esta nos sirve para clasificar los datos en falsos positivos, falsos negativos, verdaderos positivos y verdaderos negativos. Para el caso se aprecia que en su mayoria están bien predichos. 


Interpretación de las métricas:

- Accuracy_score : Corresponde a un valor porcentual dado por la razón entre el número de entradas o datos correctamente predichos respecto a la cantidad de total de estos. Se ve que es la más alta de todas. Para el caso, es la más alta de todas.

- recall : Corresponde a un valor porcentual dado por la razón entre el número de verdaderos positivos respecto a la cantidad verdaderos positivos sumado con falsos negativos. Para el caso, es la mas baja de todas.

- Precision : Corresponde a un valor porcentual dado por la razón entre el número de verdaderos positivos respecto a la cantidad verdaderos positivos sumado con falsos positivos. Para el caso, esta en un nivel intermedio respecto a las demas.

- F score : Corresponde a un valor porcentual dado por la razón entre el doble del producto de la precision con recall respecto a la suma de estos mismos. Para el caso, esta en un nivel intermedio respecto a las demas.

### 2. Cambiar algoritmo

El segundo paso es comparar con distintos modelos. Debe tener en cuenta que el modelo ocupaod resuelva el problema supervisado de clasificación.

En este punto deberá ajustar un modelo de **random forest**, aplicar las métricas y comparar con el modelo de regresión logística.

In [ ]:
# train model
rfc = RandomForestClassifier(n_estimators = 100) # algoritmo random forest
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
# metrics

y_true =  list(y_test)
y_pred = list(rfc.predict(X_test)) # predicciones con random forest


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12484     3]
 [   24   112]]

Metricas:
 
accuracy:    0.9978610472946209
recall:      0.8235294117647058
precision:   0.9739130434782609
f-score:     0.8924302788844622



Hubo un cambio considerable en la matriz de confusión y consecuentemente en las métricas, debido a la implementación del modelo de random forest que parecen indicar una mejoría en la capacidad predictiva del modelo

### 3. Técnicas de remuestreo: sobremuestreo de clase minoritaria

El tercer paso es ocupar ténicas de remuestreo, pero sobre la clase minoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase minoritaria a la clase mayoritaria.

In [ ]:
from sklearn.utils import resample

# concatenar el conjunto de entrenamiento
X = pd.concat([X_train, y_train], axis=1)

# separar las clases
not_fraud = X[X.Class==0]
fraud = X[X.Class==1]

# remuestrear  clase minoritaria
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# recombinar resultados
upsampled = pd.concat([not_fraud, fraud_upsampled])

# chequear el número de elementos por clases
upsampled.Class.value_counts()

0    37513
1    37513
Name: Class, dtype: int64

In [ ]:
# datos de entrenamiento sobre-balanceados
y_train = upsampled.Class
X_train = upsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [ ]:
upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train) # algoritmo de regresion logistica

# metrics

y_true =  list(y_test)
y_pred = list(upsampled.predict(X_test))


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12200   287]
 [   12   124]]

Metricas:
 
accuracy:    0.976313079299691
recall:      0.9117647058823529
precision:   0.30170316301703165
f-score:     0.45338208409506403



Como desventaja fue que al equiparar con técnicas de remuestreo sobre la clase minoritaria, se ven afectadas las proporciones en que se distribuyen los datos en la matriz de confusión, lo cual para este caso, significo una drástica baja en las métricas de precision y f score, considerando que los datos falsos originalmente eran bastante menos en proporción respecto a los verdaderos.  Pero a su vez, y por el mismo motivo, la ventaja se ve en que el recall subió. 

### 4. Técnicas de remuestreo - Ejemplo de clase mayoritaria

El cuarto paso es ocupar ténicas de remuestreo, pero sobre la clase mayoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase mayoritaria  a la clase minoritaria.

In [ ]:
# remuestreo clase mayoritaria
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27) # reproducible results

# recombinar resultados
downsampled = pd.concat([not_fraud_downsampled, fraud])

# chequear el número de elementos por clases
downsampled.Class.value_counts()

0    356
1    356
Name: Class, dtype: int64

In [ ]:
# datos de entrenamiento sub-balanceados

y_train = downsampled.Class
X_train = downsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [ ]:
undersampled = LogisticRegression(solver='liblinear').fit(X_train, y_train) # modelo de regresion logística

# metrics

y_true =  list(y_test)
y_pred = list(undersampled.predict(X_test))


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12216   271]
 [   16   120]]

Metricas:
 
accuracy:    0.9772637249465261
recall:      0.8823529411764706
precision:   0.3069053708439898
f-score:     0.45540796963946867



Para explicar las desventajas y ventajas, se puede extrapolar lo visto en el inciso anterior (inciso 3), puesto que las técnicas de remuestreo sobre la clase mayoritaria para equipararse a la clase minoritaria afectaron de forma similar a la matriz de confusión, lo cual puede verse reflejado también en las métricas, salvando pequeñas diferencias, puesto que al final el procedimiento fue distinto.

### 5. Conclusiones

Para finalizar el laboratorio, debe realizar un análisis comparativo con los disintos resultados obtenidos  en los pasos 1-4. Saque sus propias conclusiones del caso.

Comparativa de resultados (métricas):
 - Accuracy : Entre los pasos 1-4, se mantiene alto (cercano a 1) debido a que los datos falsos son considerablemente mas bajos que los verdaderos en general.
 - Recall : Entre los pasos 1-4, este varía al aplicar las técnicas de remuestreo debido al cambio en los valores de la diagonal principal de la matriz de confusión, presentando notables mejoras al pasar de los pasos 1-2 a los pasos 3-4.
 - Precisión : Esté varía drásticamente al comparar los pasos 1-2 contra los pasos 3-4. Su mejor alcanze está en el inciso 2.
 - f score : Al igual que el de precisión, esté varía drásticamente al comparar los pasos 1-2 contra los pasos 3-4 (aunque con valores distintos). Su mejor alcanze está en el inciso 2.

Conclusión:
Hemos podido probar que hay casos donde las métricas aplicadas pueden variar considerablemente en función de los modelos y técnicas implementadas. Claramente los dos primeros modelos se centran en predecir los datos en forma general, lo cual implica que los fraudes no son datos muy influyentes para estos. Por otro lado, se puede apreciar que los modelos 3 y 4 les dan más peso, cosa que puede verse reflejado en la disminución de la precisión y del f score, pero manteniendo alto recall y accuracy lo cual es bueno porque significa que los modelos se mantienen apegados a los datos reales.